In [2]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path=".env", override=True)

qwen_key = os.getenv('QWEN_API_KEY')

In [3]:
from langchain_tavily import TavilySearch

# 注意：新版本中类名是 TavilySearch，不是 TavilySearchResults
web_search_tool = TavilySearch(max_results=3)

In [6]:
# from langchain.prompts import PromptTemplate

# prompts = PromptTemplate(
#     input_variables=["question", "context"],
#     template="""
#     You are a helpful assistant. Answer the question based on the context provided.
#     Context: {context}
#     Question: {question}
#     Answer:
#     """,
# )

from langsmith import Client
client = Client(api_key=os.getenv('LANGSMITH_API_KEY'))
prompts = client.pull_prompt("test_langsmith:3b614a77", include_model=False)

✅ LangSmith 客户端连接成功
✅ 使用本地 PromptTemplate
提示词模板类型: <class 'langchain_core.prompts.prompt.PromptTemplate'>
输入变量: ['context', 'question']


In [11]:
from openai import OpenAI
from langsmith import traceable
from langsmith.wrappers import wrap_openai
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# 配置 ChatOpenAI（在函数外配置，避免重复创建）
llm = ChatOpenAI(
    model="qwen3-next-80b-a3b-instruct",
    openai_api_key=qwen_key,
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0.1
)

@traceable
def search(question):
    # 调用 Tavily 搜索工具
    web_docs = web_search_tool.invoke({"query": question})
    # 提取所有结果的 content 内容
    web_results = "\n".join([result['content'] for result in web_docs['results']])
    return web_results

@traceable
def explain(question, context):
    # 格式化提示词
    formatted = prompts.format(question=question, context=context)
    
    # 创建消息列表
    messages = [
        SystemMessage(content=formatted),
        HumanMessage(content=question)  # 这里应该是问题，不是格式化后的内容
    ]
    
    # 调用 ChatOpenAI
    response = llm.invoke(messages)
    
    # 返回响应内容 - 注意：ChatOpenAI 返回的是 AIMessage 对象，需要访问 .content 属性
    return response.content

@traceable
def eli5(question):
    context = search(question)
    answer = explain(question, context)    
    return answer

In [14]:
question = "Who is the president of the United States?"
print(eli5(question))

美国总统是乔·拜登。


In [ ]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "Test Langsmith"

# 是否简洁且不超过50个单词
def exact_concise(outputs: dict) -> bool:
    words = outputs["Output"].split()
    return len(words) <= 50

# 将测试集在eli5函数上运行
def run(input: dict) -> dict:
    question = input["question"]
    answer = eli5(question)
    return {"Output": answer}

evaluate(
    run,
    data=dataset_name,
    evaluators=[exact_concise],
    experiment_prefix="Test Langsmith experiment"
)

View the evaluation results for experiment: 'Test Langsmith experiment-5a1e3f74' at:
https://smith.langchain.com/o/7e2dc902-7504-4785-b065-1984ef6e8b18/datasets/93232cc8-2d06-425b-bb99-b3b1b406dcdb/compare?selectedSessions=16721794-4981-4233-904c-ec074170ce23




0it [00:00, ?it/s]

,inputs.question,outputs.Output,error,reference.output,feedback.exact_concise,execution_time,example_id,id
0,Who is the president of the United States Now?,唐纳德·约翰·特朗普,None,唐纳德·约翰·特朗普,True,2.862185,f49ec683-37c1-4293-ab13-8e71d8688fe4,1a68a8a1-5d9a-403c-ba58-9141bce4e145
1,Who is the president of the United States？,美国总统是乔·拜登。,None,唐纳德·约翰·特朗普,True,3.952093,bc91a9f5-6b11-4952-a94a-7bc03b7fdc83,da5cf27a-f691-4811-9e7d-14d80bea4347
